# Dosed Training and Evaluation

Requirements:

1. You need the data for training as h5 files, to get them you either:
  - Go through `download_and_data_format_explanation.ipynb`

    or 

  - Run `make download_example`  you can use optionnal DOWNLOAD_PATH env variable to specify an alternative directory
  
2. Have dosed installed:
    
  - Run `pip install -e .` from dosed root directory
  or 
  - Run `python setup.py develop` from dosed root directory
  


In [1]:
%matplotlib inline

In [2]:
import os
import json

h5_directory = '../data/h5'  # adapt if you used a different DOWNLOAD_PATH when running `make download_example`

# 1. Train, validation and test dataset creation

## First we select which records we want to train, validate and test on

In [3]:
import torch
import tempfile
import json
import random


from dosed.utils import Compose
from dosed.datasets import BalancedEventDataset as dataset
from dosed.models import DOSED4 as model
from dosed.datasets import get_train_validation_test
from dosed.trainers import trainers
from dosed.preprocessing import GaussianNoise, RescaleNormal, Invert

seed = 2019

In [4]:
train, validation, test = get_train_validation_test(h5_directory,
                                                    percent_test=25,
                                                    percent_validation=33,
                                                    seed=seed)

print("Number of records train:", len(train))
print("Number of records validation:", len(validation))
print("Number of records test:", len(test))

Number of records train: 11
Number of records validation: 5
Number of records test: 5


## Then we use the dataset class that will be use to generate sample for training and evaluation

- h5_directory: Location of the generic h5 files.
- signals: the signals from the h5 we want to include together with their normalization
- events: the evvents from the h5 we want to train on
- window: Spindles have a duration of ~1 seconds, so we design the samples accordingly by choosing 10 seconds windows
- ratio_positive: sample within a training batch will have a probability of "ratio_positive" to contain at least one spindle 
- n_jobs: number of process used to extract and normalize signals from h5 files.
- cache_data: cache results of extraction and normalization of signals from h5_file in h5_directory + "/.cache" (we strongly recommand to set True)



In [5]:
window = 10  # window duration in seconds
ratio_positive = 0.5  # When creating the batch, sample containing at least one spindle will be drawn with that probability

signals = [
    {"name": "eeg_raw",
     "fs": 32,
     "signals": [{"h5_paths": ["/eeg_0", "/eeg_1"],
                  "fs": 64}],
     "preprocessing": [
         {"name": "clip_and_normalize",
          "args": {
              "min_value": -150,
              "max_value": 150,
          }}
     ],
     },

    {"name": "eeg_spectrogram",
     "fs": 64,
     "signals": [{"h5_paths": ["/eeg_0", "/eeg_1"],
                  "fs": 64}],
     "preprocessing": [
         {"name": "spectrogram",
          "args": {
                  "nperseg": 8,
                  "nfft": 16,
                  "temporal_downsampling": 1,
                  "frequential_downsampling": 1,
                  "padded": True,
          }},
         {"name": "clip_and_normalize",
          "args": {
                  "min_value": 0,
                  "max_value": 200,
          }},
     ],
     }
]

events = [
    {
        "name": "spindle",
        "h5_path": "spindle",
    },
]

In [6]:
dataset_parameters = {
    "h5_directory": h5_directory,
    "signals": signals,
    "events": events,
    "window": window,
    "ratio_positive": ratio_positive,
    "n_jobs": -1,  # Make use of parallel computing to extract and normalize signals from h5
    "cache_data": True,  # by default will store normalized signals extracted from h5 in h5_directory + "/.cache" directory
}

dataset_validation = dataset(records=validation, **dataset_parameters)
dataset_test = dataset(records=test, **dataset_parameters)

# for training add data augmentation
dataset_parameters_train = {
    "transformations": Compose([
        GaussianNoise(),
        RescaleNormal(),
        Invert(),
    ])
}
dataset_parameters_train.update(dataset_parameters)
dataset_train = dataset(records=train, **dataset_parameters_train)

100%|██████████| 11/11 [00:00<00:00, 3532.45it/s]


# 2. Create a network

The main parameters for the network are:
  - default event sizes : to choose according to a priori size of the event to detect, here spindles are around 1 second
  - conv_spec_layers : layers for data as spectrogram
  - conv_raw_layers : layers for raw data

      Both are a list of layers, where each layer is a tuple :
      (channels, kernel, stride, padding, pooling)
      - channels : the number of output channels
      - kernel : the kernel to be convoluted (2D for spectrogram, 1D for raw signal)
      - stride : the stride of each convolution (2D for spectrogram, 1D for raw signal)
      - padding : the padding before the convolution (2D for spectrogram, 1D for raw signal)
      - pooling : the pooling's size (2D for spectrogram, 1D for raw signal)
  

In [7]:
default_event_sizes = [0.7, 1, 1.3]
probability_dropout = 0.1
device = torch.device("cuda")

convolution_layers = {
    "eeg_spectrogram":
        "[[4 , (1,3), (1,2), (0,2), (2,2)]] + " + \
        "[[8 , (1,3), (1,2), (0,2), (2,2)]] + " + \
        "[[16, (1,3), (1,1), (0,2), (2,2)]] + " + \
        "[[32, (1,3), (1,1), (0,2), (1,2)]] + " + \
        "[[64, (1,3), (1,1), (0,2), (1,2)]]",
        #(channels, kernel, stride, padding, pooling)
    "eeg_raw":
        "[[4 , (5,), (1,), (2,), (2,)]] + " + \
        "[[8 , (5,), (1,), (2,), (2,)]] + " + \
        "[[16 , (5,), (1,), (2,), (2,)]] + " + \
        "[[32, (5,), (1,), (2,), (2,)]] + " + \
        "[[64, (5,), (1,), (2,), (2,)]]"
        #(channels, kernel, stride, padding, pooling)
}

In [8]:
sampling_frequency = dataset_train.fs

net_parameters = {
    "detection_parameters": {
        "overlap_non_maximum_suppression": 0.5,
        "classification_threshold": 0.7
    },
    "default_event_sizes": default_event_sizes,
    "convolution_layers": convolution_layers,
    "pdrop": probability_dropout,
    "fs": sampling_frequency,   # just used to print architecture info with right time
    "input_shapes": dataset_train.input_shapes,
    "window": window,
    "number_of_classes": dataset_train.number_of_classes,
}
net = model(**net_parameters)
net = net.to(device)

# 3. Train the network

Parameters are
  - learning_rate
  - loss type

In [9]:
optimizer_parameters = {
    "lr": 5e-3,
    "weight_decay": 1e-8,
}
loss_specs = {
    "type": "focal",
    "parameters": {
        "number_of_classes": dataset_train.number_of_classes,
        "device": device,
    }
}
epochs = 20

In [ ]:
trainer = trainers["adam"](
    net,
    optimizer_parameters=optimizer_parameters,
    loss_specs=loss_specs,
    epochs=epochs,
)

best_net_train, best_metrics_train, best_threshold_train = trainer.train(
    dataset_train,
    dataset_validation,
)

  0%|          | 0/20 [00:00<?, ?it/s]

Device:  cuda:0
Logging data to /tmp/tmp7kjltqmv/train_history.json


 10%|█         | 2/20 [05:44<56:57, 189.88s/it, best_metric_score=0.0405, last_update=0, threshold=0.605]

# 4. Predict

In [ ]:
predictions = best_net_train.predict_dataset(
    dataset_test,
    best_threshold_train,
)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

record = dataset_test.records[1]

index_spindle = 30
window_duration = 5

# retrive spindle at the right index
spindle_start = float(predictions[record][0][index_spindle][0]) / sampling_frequency
spindle_end = float(predictions[record][0][index_spindle][1]) / sampling_frequency

# center data window on annotated spindle 
start_window = spindle_start + (spindle_end - spindle_start) / 2 - window_duration
stop_window = spindle_start + (spindle_end - spindle_start) / 2 + window_duration

# Retrieve EEG data at right index
index_start = int(start_window * sampling_frequency)
index_stop = int(stop_window * sampling_frequency)
y = dataset_test.signals[record]["data"]["raw"][0][index_start:index_stop]

# Build corresponding time support
t = start_window + np.cumsum(np.ones(index_stop - index_start) * 1 / sampling_frequency)

plt.figure(figsize=(16, 5))
plt.plot(t, y)
plt.axvline(spindle_end)
plt.axvline(spindle_start)
plt.ylim([-1, 1])
plt.show()